# Trabajo Práctico Nº3 - Big Data

### Mateo Servent, Joaquín Musich y Andres Cuellas

### Parte I: Análisis de la base de hogares y cálculo de pobreza

In [6]:
import pandas as pd

# Leer la base de datos
df = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP3/usu_hogar_T123.xlsx')

# Estadísticos descriptivos (más la exploración del libro de codigos)
 
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ANO4,16815.0,2023.000000,0.000000,2023.0,2023.0,2023.0,2023.0,2023.0
TRIMESTRE,16815.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
NRO_HOGAR,16815.0,1.064347,1.177592,1.0,1.0,1.0,1.0,52.0
REALIZADA,16815.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
REGION,16815.0,35.439191,15.237237,1.0,40.0,42.0,43.0,44.0
...,...,...,...,...,...,...,...,...
VII1_2,16815.0,0.590009,2.756843,0.0,0.0,0.0,0.0,97.0
VII2_1,16815.0,70.012608,43.469641,1.0,3.0,98.0,98.0,98.0
VII2_2,16815.0,0.403449,3.397350,0.0,0.0,0.0,0.0,98.0
VII2_3,16815.0,0.103836,1.289136,0.0,0.0,0.0,0.0,98.0


### A partir de un análisis preliminar, consideramos que posibles determinantes claves en la predicción de la pobreza dentro del conjunto de datos de hogares serían los atributos vinculados a la infraestructura habitacional (calidad estructural, conectividad a servicios básicos, instalaciones sanitarias), el esquema de propiedad o alquiler del inmueble y la distribución demográfica por edad en el núcleo familiar. Estos indicadores, podrían llegar a ser considerados esenciales para una estimación más afinada.

In [24]:
# 2)
# Filtrar observaciones
df_filtrado = df[df['AGLOMERADO'].isin([32, 33])]

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,GDECCFR,PDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORUYHLMSMCDEIJAH00719364,1,1,1,1,2,1957-10-17 00:00:00,65,1,1.0,...,5.0,NaN,6,2540,1,0,2,0,0,0
1,TQRMNORUYHLMSMCDEIJAH00719364,1,2,1,2,1,1952-04-07 00:00:00,70,1,1.0,...,5.0,NaN,6,2540,1,0,2,0,0,0
2,TQRMNOSRQHJNSOCDEIJAH00802640,1,1,1,1,1,1986-09-26 00:00:00,36,1,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
3,TQRMNOSRQHJNSOCDEIJAH00802640,1,2,1,2,2,1994-05-02 00:00:00,28,1,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
4,TQRMNOSRQHJNSOCDEIJAH00802640,1,3,0,3,2,2014-03-20 00:00:00,9,5,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,TQRMNOPVVHKLRQCDEIJAH00780546,1,3,0,3,1,2013-08-05 00:00:00,9,5,2.0,...,9.0,NaN,9,2295,1,2,98,0,0,0
7615,TQRMNOQPTHKKRRCDEIJAH00780327,1,1,1,1,2,1945-06-05 00:00:00,77,3,1.0,...,10.0,NaN,10,1516,1,0,96,0,0,0
7616,TQRMNOPTXHKORSCDEIJAH00780269,1,1,1,1,2,1977-07-31 00:00:00,45,5,1.0,...,12.0,NaN,12,0,1,2,3,0,0,0
7617,TQRMNOPTXHKORSCDEIJAH00780269,1,2,1,3,1,2005-06-17 00:00:00,17,5,1.0,...,12.0,NaN,12,0,1,2,3,0,0,0


In [10]:
# 3)

dfI = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP2/EPH_usu_1er_Trim_2023/usu_individual_T123.xlsx')

dfI = dfI[dfI['AGLOMERADO'].isin([32, 33])]

# Definimos las intersección entre las dos bases como una lista
columnas_duplicadas = set(dfI.columns).intersection(set(df_filtrado.columns))
# Removemos CODUSU y NRO_HOGAR de la lista ya que queremos usarlas para la intersección 
#(y las columnas sobre las que se une no generan duplicados)
columnas_duplicadas.remove("CODUSU")
columnas_duplicadas.remove("NRO_HOGAR")

# Hacemos el merge habiendo dropeado las columnas repetidas
df2 = pd.merge(
    dfI.drop(columnas_duplicadas, axis=1), 
    df_filtrado,
    on= ['NRO_HOGAR', 'CODUSU'])

In [25]:
# 4 y 5 ) 
def eliminar_categoria_9(df, column_list):
    """
    Elimina la categoría con valor 9 de una serie de variables en un DataFrame.
    
    Parámetros:
    - df: DataFrame donde se encuentran las columnas.
    - column_list: Lista de columnas en las que se quiere eliminar la categoría con valor 9.
    
    Retorna:
    - DataFrame con las categorías con valor 9 eliminadas en las columnas especificadas.
    """
    for column in column_list:
        df = df[df[column] != 9]
    return df


df3 = eliminar_categoria_9(df2, ['NIVEL_ED', 'CH08', 'CH07', 'CAT_OCUP', 'CH15', 'CH16']) 
# Hay más NAs en otras variables
df4 = df3[(df3['CH06'] > 0)] # Edad menor a cero

In [22]:
def columnas_duplicadas(df):
    """
    Esta función verifica si hay columnas con valores idénticos en un DataFrame.
    
    Argumentos:
    - df: DataFrame de pandas
    
    Retorna:
    - Una lista de tuplas con los nombres de las columnas repetidas.
    """
    
    columnas_repetidas = []
    
    for i in range(df.shape[1]):
        for j in range(i+1, df.shape[1]):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                columnas_repetidas.append((df.columns[i], df.columns[j]))
    
    return columnas_repetidas


def eliminar_columnas_duplicadas(df):
    # Obtener las columnas duplicadas
    cols_duplicadas = columnas_duplicadas(df)
    
    # Crear una lista de columnas para eliminar
    columnas_para_eliminar = [col[1] for col in cols_duplicadas]
    
    # Eliminar las columnas
    df = df.drop(columns=columnas_para_eliminar)
    
    return df

In [28]:
eliminar_columnas_duplicadas(df4) # verificamos si hubiera columnas duplicadas y las eliminamos

def drop_high_na(df, threshold=0.7):
    """
    Elimina las columnas y las filas del DataFrame que tienen un porcentaje de valores faltantes 
    superior al umbral especificado.

    Parámetros:
    - df: DataFrame de entrada.
    - threshold: umbral de porcentaje de valores faltantes para eliminar una columna o fila (valor predeterminado = 0.7).

    Retorna:
    - DataFrame después de eliminar las columnas y filas con un alto porcentaje de valores faltantes.
    """
    # Calcula el porcentaje de valores faltantes para cada columna
    missing_percentage_col = df.isnull().mean()
    
    # Filtra las columnas que tienen un porcentaje de valores faltantes superior al umbral
    columns_to_drop = missing_percentage_col[missing_percentage_col > threshold].index
    
    # Elimina las columnas seleccionadas del DataFrame
    df_cleaned = df.drop(columns=columns_to_drop)
    
    # Calcula el porcentaje de valores faltantes para cada fila
    missing_percentage_row = df_cleaned.isnull().mean(axis=1)
    
    # Filtra las filas que tienen un porcentaje de valores faltantes superior al umbral
    rows_to_drop = missing_percentage_row[missing_percentage_row > threshold].index
    
    # Elimina las filas seleccionadas del DataFrame
    df_cleaned = df_cleaned.drop(index=rows_to_drop)
    
    return df_cleaned

drop_high_na(df4)

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,RDECCFR,GDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORUYHLMSMCDEIJAH00719364,1,1,1,1,2,1957-10-17 00:00:00,65,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
1,TQRMNORUYHLMSMCDEIJAH00719364,1,2,1,2,1,1952-04-07 00:00:00,70,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
2,TQRMNOSRQHJNSOCDEIJAH00802640,1,1,1,1,1,1986-09-26 00:00:00,36,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
3,TQRMNOSRQHJNSOCDEIJAH00802640,1,2,1,2,2,1994-05-02 00:00:00,28,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
4,TQRMNOSRQHJNSOCDEIJAH00802640,1,3,0,3,2,2014-03-20 00:00:00,9,5,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,TQRMNOPVVHKLRQCDEIJAH00780546,1,3,0,3,1,2013-08-05 00:00:00,9,5,2.0,...,9,9.0,9,2295,1,2,98,0,0,0
7615,TQRMNOQPTHKKRRCDEIJAH00780327,1,1,1,1,2,1945-06-05 00:00:00,77,3,1.0,...,10,10.0,10,1516,1,0,96,0,0,0
7616,TQRMNOPTXHKORSCDEIJAH00780269,1,1,1,1,2,1977-07-31 00:00:00,45,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0
7617,TQRMNOPTXHKORSCDEIJAH00780269,1,2,1,3,1,2005-06-17 00:00:00,17,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0


In [33]:
# 6) Estadísticas descriptivas
# IV6: Suministro de agua
# IV3: Tipos de piso interior
# IV4: Tipo de cubierta exterior del techo
# IV11: Desague del baño
# IV12_3: vivienda ubicada en una villa de emergencia

df4[['IV3', "IV12_3", 'V4', 'IV6', 'IV11']].describe().T

,count,mean,std,min,25%,50%,75%,max
IV3,7536.0,1.123275,0.347611,1.0,1.0,1.0,1.0,4.0
IV12_3,7536.0,1.994294,0.075327,1.0,2.0,2.0,2.0,2.0
V4,7536.0,2.004910,0.248904,1.0,2.0,2.0,2.0,9.0
IV6,7536.0,1.026407,0.169994,1.0,1.0,1.0,1.0,3.0
IV11,7536.0,1.372346,0.642366,0.0,1.0,1.0,2.0,4.0


In [53]:
# 7) 
adultos_data = pd.read_excel("/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP2/tabla_adulto_equiv.xlsx", skiprows=3)
adultos_data = adultos_data.dropna(how='all')
adultos_data = adultos_data.iloc[:23, :]

def categorize_age(age):
    if age == 0:
        return "Menor de 1 año"
    elif age == 1:
        return "1 año"
    elif age == 2:
        return "2 años"
    elif age == 3:
        return "3 años"
    elif age == 4:
        return "4 años"
    elif age == 5:
        return "5 años"
    elif age == 6:
        return "6 años"
    elif age == 7:
        return "7 años"
    elif age == 8:
        return "8 años"
    elif age == 9:
        return "9 años"
    elif age == 10:
        return "10 años"
    elif age == 11:
        return "11 años"
    elif age == 12:
        return "12 años"
    elif age == 13:
        return "13 años"
    elif age == 14:
        return "14 años"
    elif age == 15:
        return "15 años"
    elif age == 16:
        return "16 años"
    elif age == 17:
        return "17 años"
    elif 18 <= age <= 29:
        return "18 a 29 años"
    elif 30 <= age <= 45:
        return "30 a 45 años"
    elif 46 <= age <= 60:
        return "46 a 60 años"
    elif 61 <= age <= 75:
        return "61 a 75 años"
    else:
        return "más de 75 años"

# Creando la columna "Rango de Edad"
df4 = df4.copy()
df4.loc[:, 'Rango de Edad'] = df4['CH06'].apply(categorize_age)

# Creando dos dataframes: uno para Mujeres y otro para Varones
df_mujeres = adultos_data[['Edad', 'Mujeres']].copy()
df_mujeres['CH04'] = 2
df_mujeres.rename(columns={'Mujeres': 'adulto_equiv'}, inplace=True)

df_varones = adultos_data[['Edad', 'Varones']].copy()
df_varones['CH04'] = 1
df_varones.rename(columns={'Varones': 'adulto_equiv'}, inplace=True)

# Concatenando ambos dataframes
adultos_equiv_data = pd.concat([df_mujeres, df_varones], axis=0)

# Reordenando las columnas y ordenando el dataframe por Edad
adultos_equiv_data = adultos_equiv_data[['Edad', 'CH04', 'adulto_equiv']]
adultos_equiv_data = adultos_equiv_data.sort_values(by=['Edad', 'CH04']).reset_index(drop=True)

In [57]:
# Renombrando la columna "Rango de Edad" a "Edad" en df4
df4.rename(columns={'Rango de Edad': 'Edad'}, inplace=True)

# Uniendo df4 con adultos_equiv_data usando las columnas "Edad" y "CH04"
df5 = pd.merge(df4, adultos_equiv_data[['Edad', 'CH04', 'adulto_equiv']],
               on=['Edad', 'CH04'], how='left')

# Mostrando las primeras filas de la base resultante
df5[['CH04', 'CH06', 'Edad', 'adulto_equiv']].head()

# Agrupar por NRO_HOGAR y CODUSU y calcular la suma de adultos_equiv para cada combinación
sum_adulto_equiv = df5.groupby(['NRO_HOGAR', 'CODUSU'])['adulto_equiv'].sum().reset_index(name='ad_equiv_hogar')

# Fusionar con df5 para asignar los valores calculados a la columna ad_equiv_hogar
df6 = pd.merge(df5, sum_adulto_equiv, on=['NRO_HOGAR', 'CODUSU'], how='left')
df6 = df6.copy()

# Mostrar las primeras 10 filas del DataFrame para verificar
df6[['NRO_HOGAR', 'CODUSU', 'adulto_equiv', 'ad_equiv_hogar']].head(10)

,NRO_HOGAR,CODUSU,adulto_equiv,ad_equiv_hogar
0,1,TQRMNORUYHLMSMCDEIJAH00719364,0.67,1.50
1,1,TQRMNORUYHLMSMCDEIJAH00719364,0.83,1.50
2,1,TQRMNOSRQHJNSOCDEIJAH00802640,1.00,2.45
3,1,TQRMNOSRQHJNSOCDEIJAH00802640,0.76,2.45
4,1,TQRMNOSRQHJNSOCDEIJAH00802640,0.69,2.45
5,1,TQRMNOSWQHJLQRCDEIJAH00796254,1.00,2.00
6,1,TQRMNOSWQHJLQRCDEIJAH00796254,1.00,2.00
7,1,TQRMNOPYVHLMRLCDEIJAH00719346,1.00,2.78
8,1,TQRMNOPYVHLMRLCDEIJAH00719346,0.76,2.78
9,1,TQRMNOPYVHLMRLCDEIJAH00719346,1.02,2.78


In [88]:
# 8) División entre quienes reportaron ingresos y entre quienes no lo hicieron
# Ingreso neesario según canasta básica

norespondieron = df6[(df6['DECIFR'] == 12) | (df6['DECIFR'] == 13) & (df6['ITF'] == 0)].copy()
respondieron = df6[(df6['DECIFR'] != 12) & (df6['DECIFR'] != 13) & (df6['ITF'] != 0)].copy()

canasta_basica_total = 57371.05

respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * canasta_basica_total
norespondieron['ingreso_necesario'] = norespondieron['ad_equiv_hogar'] * canasta_basica_total

#9) 
# Variable de conteo casos divididos por la línea de pobreza

respondieron['pobre'] = (respondieron['ITF'] < respondieron['ingreso_necesario']).astype(int)

In [89]:
# 10) 
# Filtrar solo los hogares que son pobres
hogares_pobres = respondieron[respondieron['pobre'] == 1]

In [91]:
# Sumar el ponderador PONDIH para los hogares únicos
total_hogares_ponderados = respondieron.drop_duplicates(subset=['CODUSU', 'NRO_HOGAR'])['PONDIH'].sum()

# Sumar el ponderador PONDIH para los hogares únicos que son pobres
hogares_pobres_ponderados = hogares_pobres.drop_duplicates(subset=['CODUSU', 'NRO_HOGAR'])['PONDIH'].sum()

# Calcular el porcentaje de hogares pobres ponderados en relación al total ponderado
porcentaje_hogares_pobres_ponderados = (hogares_pobres_ponderados / total_hogares_ponderados) * 100

print(f'Número ponderado de hogares identificados como pobres: {hogares_pobres_ponderados}')
print(f'Porcentaje ponderado de hogares identificados como pobres: {porcentaje_hogares_pobres_ponderados:.2f}%')

Número ponderado de hogares identificados como pobres: 1499962
Porcentaje ponderado de hogares identificados como pobres: 28.24%


### La diferencia con los valores que reporta INDEC (30,3%), tiene que ver con el tratamiento que dieron a valores faltantes y al no reporte de ingresos. Recordamos que en nuestro caso eliminamos casos donde el 70% de los valores de la fila o columna eran NAs y, además, solo estamos considerando quienre reportaron ingresos. 

# Parte II: Construcción de funciones

In [316]:
# 1)
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

def evalua_modelo(x_train, y_train, x_test, y_test, tipos=['logistico', 'LDA', 'kNN'], lambda_=1.0, k_neighbors=5):
    # Estandarizar datos
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    # DataFrame para almacenar resultados
    resultados = pd.DataFrame({
        'Metrica': ['AUC', 'Accuracy', 'ECM', 'True P', 'False P', 'True N', 'False N'],
        'Lambda': [np.nan] * 7,  # Columna vacía para lambda
        'k_neighbors': [np.nan] * 7  # Columna vacía para k_neighbors
    })

    for tipo in tipos:
        # Ajustar el modelo y establecer el hiperparámetro
        if tipo == 'logistico':
            modelo = LogisticRegression(C=1/lambda_, max_iter=10000).fit(x_train, y_train)
            resultados['Lambda'] = lambda_
        elif tipo == 'LDA':
            modelo = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto').fit(x_train, y_train)
        elif tipo == 'kNN':
            modelo = KNeighborsClassifier(n_neighbors=k_neighbors).fit(x_train, y_train)
            resultados['k_neighbors'] = k_neighbors
        else:
            raise ValueError(f"Tipo de modelo {tipo} no reconocido.")
        
        # Predicciones
        y_pred = modelo.predict(x_test)
        y_pred_proba = modelo.predict_proba(x_test)[:, 1]
        
        # Métricas
        conf_matrix = confusion_matrix(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        acc = accuracy_score(y_test, y_pred)
        ecm = mean_squared_error(y_test, y_pred)
        
        # Almacenar resultados en el DataFrame
        resultados[f'Valor_{tipo}'] = [auc, acc, ecm, 
                                      conf_matrix[1, 1], conf_matrix[0, 1],
                                      conf_matrix[0, 0], conf_matrix[1, 0]]

    # Redondear los valores a dos decimales
    for col in resultados.columns[1:]:
        resultados[col] = resultados[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

    return resultados

In [318]:
from sklearn.impute import SimpleImputer

# Separar las características (X) y la variable dependiente (y)
X = respondieron.drop('pobre', axis=1)
X = X.select_dtypes(include=["number"])  # mantenemos las variables numéricas

# Eliminar las columnas que solo contienen NaN
X = X.dropna(axis=1, how='all')

# Imputar datos faltantes con la media, es una medida conroversial. Permite continuar con las predicciones sin 
# perder mucha información, pero se atribuye valores incorrectos a casos que no reportan información.
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)

# Asegurarse de que y tenga las mismas filas que X_imputed
y_cleaned = respondieron.loc[X_imputed.index, 'pobre']

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y_cleaned, test_size=0.3, random_state=2023)


print("Conjunto de entrenamiento:", "X_train =", len(x_train), "Y_train =", len(y_train))
print("Conjunto de prueba:", "X_test =", len(x_test), "Y_test =", len(y_test))

evalua_modelo(x_train, y_train, x_test, y_test) # evalua los 3 pero se podría seleccionar solo 1 o 2

Conjunto de entrenamiento: X_train = 2919 Y_train = 2919
Conjunto de prueba: X_test = 1252 Y_test = 1252


,Metrica,Lambda,k_neighbors,Valor_logistico,Valor_LDA,Valor_kNN
0,AUC,1.0,5,0.996,0.989,0.928
1,Accuracy,1.0,5,0.963,0.950,0.860
2,ECM,1.0,5,0.037,0.050,0.140
3,True P,1.0,5,415.000,429.000,332.000
4,False P,1.0,5,15.000,45.000,61.000
5,True N,1.0,5,791.000,761.000,745.000
6,False N,1.0,5,31.000,17.000,114.000


In [319]:
# 2) 
from sklearn.model_selection import StratifiedKFold

def cross_validation(X, y, k=10, modelos=['logistico', 'LDA', 'kNN']):
    # El valor predeterminado para k particiones es 10
    # modelos es una lista de los tipos de modelos a evaluar, por defecto se evalúan los tres.
    
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2023) 
    # subconjuntos estratificados mejoran las particiones y dan mayor estabilidad a las métricas
    resultados_totales = []

    for train_index, test_index in skf.split(X, y):  # pasamos 'y' también a skf.split
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Evaluamos todos los modelos especificados en la lista modelos
        resultados = evalua_modelo(X_train, y_train, X_test, y_test, modelos)
        resultados_totales.append(resultados)

    # Consolidar los resultados en un único DataFrame
    resultados_df = pd.concat(resultados_totales, axis=0).reset_index(drop=True)
    
    # Añadir una columna para identificar el fold
    resultados_df['Fold'] = [i+1 for i in range(k) for _ in range(len(resultados))]
    
    # Reordenar las columnas para que 'Fold' aparezca al principio
    cols = resultados_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    resultados_df = resultados_df[cols]

    return resultados_df

cross_validation(X_imputed, y_cleaned)

,Fold,Metrica,Lambda,k_neighbors,Valor_logistico,Valor_LDA,Valor_kNN
0,1,AUC,1.0,5,0.998,0.990,0.950
1,1,Accuracy,1.0,5,0.978,0.957,0.892
2,1,ECM,1.0,5,0.022,0.043,0.108
3,1,True P,1.0,5,152.000,152.000,128.000
4,1,False P,1.0,5,6.000,15.000,18.000
...,...,...,...,...,...,...,...
65,10,ECM,1.0,5,0.019,0.053,0.103
66,10,True P,1.0,5,150.000,150.000,122.000
67,10,False P,1.0,5,3.000,17.000,10.000
68,10,True N,1.0,5,259.000,245.000,252.000


In [339]:
# 3) Integramos la dos funciones pasadas 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, mean_squared_error
import numpy as np

def evalua_config(X, y, lambda_values=[1.0], k_neighbors_values=[5], k=10, modelos=['logistico', 'LDA', 'kNN']):
    # Lista para almacenar los resúmenes de todas las combinaciones
    resumenes_totales = []
    
    # Iterar sobre cada valor de lambda y k_neighbors
    for lambda_ in lambda_values:
        for k_neighbors in k_neighbors_values:
            
            # Realizar la validación cruzada
            resultados_df = cross_validation(X, y, k=k, modelos=modelos)
            
            # Evaluar el ECM mínimo y extraer la información correspondiente
            metodos = ['Valor_logistico', 'Valor_LDA', 'Valor_kNN']
            resumen = []
            
            for metodo in metodos:
                min_ecm = resultados_df[resultados_df['Metrica'] == 'ECM'][metodo].min()
                row = resultados_df[(resultados_df['Metrica'] == 'ECM') & (resultados_df[metodo] == min_ecm)]
                
                if 'logistico' in metodo:
                    resumen.append({
                        'Metodo': metodo,
                        'ECM_minimo': min_ecm,
                        'Lambda': lambda_,
                        'Fold': row['Fold'].values[0],
                        'k_neighbors': np.nan
                    })
                elif 'kNN' in metodo:
                    resumen.append({
                        'Metodo': metodo,
                        'ECM_minimo': min_ecm,
                        'Lambda': np.nan,
                        'Fold': row['Fold'].values[0],
                        'k_neighbors': k_neighbors
                    })
                else:
                    resumen.append({
                        'Metodo': metodo,
                        'ECM_minimo': min_ecm,
                        'Lambda': np.nan,
                        'Fold': row['Fold'].values[0],
                        'k_neighbors': np.nan
                    })

            # Convertir el resumen actual a DataFrame y agregarlo a la lista de resúmenes
            resumen_df = pd.DataFrame(resumen)
            resumenes_totales.append(resumen_df)

    # Consolidar todos los resúmenes en un único DataFrame
    resumen_total = pd.concat(resumenes_totales, axis=0).reset_index(drop=True)
    resumen_total = resumen_total.drop_duplicates().reset_index(drop=True)

    return resumen_total

evalua_config(X_imputed, y_cleaned, lambda_values=[1, 10], k_neighbors_values=[3, 5])

,Metodo,ECM_minimo,Lambda,Fold,k_neighbors
0,Valor_logistico,0.012,1.0,7,NaN
1,Valor_LDA,0.041,NaN,8,NaN
2,Valor_kNN,0.103,NaN,10,3.0
3,Valor_kNN,0.103,NaN,10,5.0
4,Valor_logistico,0.012,10.0,7,NaN


In [340]:
# 4)

def evalua_multiples_metodos(X, y, lambdas=[1.0], k_neighbors_values=[5], k=10, modelos=['logistico', 'LDA', 'kNN']):
    resumenes = []
    
    # Iterar sobre cada combinación de valores de lambda y k_neighbors
    for lambda_ in lambdas:
        for k_neighbors in k_neighbors_values:
            resumen_df = evalua_config(X, y, lambda_=lambda_, k_neighbors=k_neighbors, k=k, modelos=modelos)
            resumenes.append(resumen_df)
    
    # Consolidar todos los resúmenes en un único DataFrame
    resumen_total = pd.concat(resumenes, axis=0).reset_index(drop=True)
    
    # Identificar el ECM mínimo para cada método
    metodos = ['Valor_logistico', 'Valor_LDA', 'Valor_kNN']
    mejores_resultados = []
    
    for metodo in metodos:
        min_ecm = resumen_total[resumen_total['Metodo'] == metodo]['ECM_minimo'].min()
        row = resumen_total[(resumen_total['Metodo'] == metodo) & (resumen_total['ECM_minimo'] == min_ecm)]
        mejores_resultados.append(row)
    
    mejores_df = pd.concat(mejores_resultados, axis=0).reset_index(drop=True)
    
    # Eliminar filas duplicadas
    mejores_df = mejores_df.drop_duplicates().reset_index(drop=True)
    
    return mejores_df


evalua_multiples_metodos(X_imputed, y_cleaned, lambdas=[1, 5, 10], k_neighbors_values=[3, 5, 7])

TypeError: evalua_config() got an unexpected keyword argument 'lambda_'